Importando as bibliotecas necessárias.

In [28]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import requests
import pandas as pd

Extração dos links, utilizando BS4 e Selenium, site base contém diversos links dos imóveis e um botão "Ver Mais"
em que utilizei Selenium (linha 20 - 22) para continuar avançando.

Nota: Pode ser útil alterar os parãmetros em 'WebDriverWait(driver, 2)' e 'time.sleep(2)' dependendo da potência
do computador do usuário desse código.

In [29]:
lista_links = []

def extrair_links(soup):
    linked = soup.find_all("a", class_='sc-1d0oyoa-0 bfDoiv')
    return [link.get('href') for link in linked]

url = f'https://www.quintoandar.com.br/comprar/imovel/rio-de-janeiro-rj-brasil'

driver = webdriver.Chrome()
chrome_options = Options()
chrome_options.add_argument('--headless')

driver = webdriver.Chrome(options=chrome_options)

try:
    driver.get(url)
    wait = WebDriverWait(driver, 2)
    for i in range(5):
        try:
            ver_mais_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[text()="Ver mais"]')))
            ver_mais_button.click()
            time.sleep(2)
        except:
            break
    final_html = driver.page_source
    soup_final = BeautifulSoup(final_html, 'html.parser')
    lista_links = extrair_links(soup_final)

finally:
    driver.quit()

print(lista_links)

['https://www.quintoandar.com.br/imovel/894481185/comprar/apartamento-2-quartos-meier-rio-de-janeiro', 'https://www.quintoandar.com.br/imovel/893153438/comprar/apartamento-1-quarto-rio-comprido-rio-de-janeiro', 'https://www.quintoandar.com.br/imovel/894462586/comprar/casa-4-quartos-tanque-rio-de-janeiro', 'https://www.quintoandar.com.br/imovel/893727119/comprar/apartamento-2-quartos-tanque-rio-de-janeiro', 'https://www.quintoandar.com.br/imovel/893639865/comprar/apartamento-2-quartos-vila-isabel-rio-de-janeiro', 'https://www.quintoandar.com.br/imovel/894479919/comprar/apartamento-2-quartos-estacio-rio-de-janeiro', 'https://www.quintoandar.com.br/imovel/893963229/comprar/casa-4-quartos-piedade-rio-de-janeiro', 'https://www.quintoandar.com.br/imovel/894006088/comprar/apartamento-2-quartos-tanque-rio-de-janeiro', 'https://www.quintoandar.com.br/imovel/893769478/comprar/apartamento-3-quartos-jacarepagua-rio-de-janeiro', 'https://www.quintoandar.com.br/imovel/894443612/comprar/casa-3-quarto

Criando as listas em que os dados serão armazenados.

In [30]:
lista_preços = []
lista_elem = []
lista_item = []
lista_condominio = []
lista_id = []
lista_iptus = []
lista_endereço1 = []
lista_endereço2 = []
lista_tempos = []
lista_tempos_dias = []
lista_tipos = []

Criando as funções que serão utilizadas.

In [31]:
def id(soup):
    buscando = soup.find_all('a',{'aria-current': 'page'})
    for encontrado in buscando:
        id = encontrado.find('span', class_ = 'CozyTypography xih2fc Bm9DLt _1tBHcU')
        if id:
            lista_id.append(id.text.strip().replace("Imóvel",""))
        else:
            lista_id.append("Not Informed")
    return lista_id

def tipo(soup):
    buscando = soup.find_all('h1', class_ = 'CozyTypography xih2fc EKXjIf wIyEP2 _8JKqPG r4Q8xM')
    for encontrado in buscando:
        cleaning = encontrado.text.split()
        lista_tipos.append(cleaning[0] if cleaning else "Not Informed")
    return lista_tipos

def precos(soup):
    buscando = soup.find("p", class_='CozyTypography xih2fc wIyEP2 _8JKqPG r4Q8xM')
    if buscando:
        for elem in buscando:
            cleaning = elem.text.strip().replace("\xa0","")
            lista_preços.append(cleaning.replace("R$",''))
    else:
        lista_preços.append("Not Informed")
    return lista_preços

def enderecos_pt1(soup):
    buscando = soup.find_all("small", class_='CozyTypography xih2fc pwAPLE')
    for endereço in buscando:
        lista_endereço1.append(endereço.text.strip().replace("\xa0",""))
    return lista_endereço1

def enderecos_pt2(soup):
        buscando = soup.find("h4", class_='CozyTypography xih2fc EqjlRj')
        if buscando:
                lista_endereço2.append(buscando.text.strip())
        else:
                lista_endereço2.append("Not Informed")
        return lista_endereço2

def elementos_base(soup):
        cont = 0
        elemexists = soup.find_all("p", class_ = "CozyTypography xih2fc EKXjIf Ci-jp3")
        if elemexists:
            for elem_end in elemexists:
                if cont == 7:
                    break
                else:
                    lista_elem.append(elem_end.text.strip().replace("\xa0",""))
                    cont = cont + 1
        else:
            for i in range(7):
                lista_elem.append("Not Informed")
        return lista_elem

def itens(soup):
    sublist = []
    item = soup.find_all("div", class_='MuiBox-root mui-117mro9')
    if item:
        for elem in item:
            exists = elem.find_all("p", class_ = 'CozyTypography xih2fc Ci-jp3 p4uM-0')
            if exists:
                for elemnow in exists:
                    sublist.append(elemnow.text.strip().replace("\xa0",""))
    else:
        sublist.append("Not Informed")
    cleaning = ', '.join(sublist)
    return cleaning

def condominio(soup):
    buscando = soup.find('span', class_ = 'CozyTypography sc-1jl0yev-0 cTsiPw xih2fc _72Hu5c Ci-jp3')
    if buscando:
        for custocondominio in buscando:
            cleaning = custocondominio.text.strip().replace('\xa0','')
            lista_condominio.append(cleaning.replace("R$",""))
    else:
        lista_condominio.append("Not Informed")
    return lista_condominio

def iptu(soup):
    buscando = soup.find_all('span', class_="CozyTypography sc-1jl0yev-0 cTsiPw xih2fc _72Hu5c Ci-jp3")
    if buscando:
        for i in range(1, len(buscando), 2):
            iptu_text = buscando[i].text
            cleaned = iptu_text.replace("R$", "").replace("\xa0", "")
            cleaned1 = cleaned.replace("12x ","")
            lista_iptus.append(cleaned1)
    else:
        lista_iptus.append("Not Informed")
    return lista_iptus

def tempo(soup):
    buscando = soup.find('small',class_ = 'CozyTypography xih2fc Bm9DLt _72Hu5c _1tBHcU')
    if buscando:
        lista_tempos.append(buscando.text.strip())
    else:
        lista_tempos.append("Not Informed")
    return lista_tempos

def converte_dias(lista_tempos):
    for elem in lista_tempos:
        divisão = elem.split()
        if elem == "Not Informed":
            lista_tempos_dias.append("Not Informed")
        elif divisão[3] == 'mês' or divisão[3] == 'meses':
            lista_tempos_dias.append(int(divisão[2]) * 30)
        elif divisão[3] == 'horas' or divisão[3] == 'hora':
            lista_tempos_dias.append(round(int(divisão[2])/24,2))
        elif divisão[3] == 'dias' or divisão[3] == 'dia':
            lista_tempos_dias.append(int(divisão[2]))
        else:
            lista_tempos_dias.append(int(divisão[4])*365)
    return lista_tempos_dias

Iterando pela lista de links obtidos, e aplicando as funções mencionadas acima.

In [32]:
for links in lista_links:
    response = requests.get(links)
    html_content = response.content
    soup = BeautifulSoup(html_content, 'html.parser')
    iptu(soup)
    id(soup)
    precos(soup)
    condominio(soup)
    elementos_base(soup)
    enderecos_pt1(soup)
    enderecos_pt2(soup)
    lista_item.append(itens(soup))
    tempo(soup)
    tipo(soup)

Populando mais listas derivadas da outra lista "Lista_Elem", que foi preparada de modo que, a cada 7 elementos
fosse relacionada à um novo link, além disso, também convertendo os "meses", "anos", "horas para "dias".

In [ ]:
quartos = []
tamanho = []
metro = []
vagas = []
banheiros = []
andar = []
mobilia = []

for i in range(len(lista_elem)):
    if i % 7 == 0:
        quartos.append(lista_elem[i][0])
    elif i % 7 == 1:
        tamanho.append(lista_elem[i].replace("m²",''))
    elif i % 7 == 2:
        metro.append(lista_elem[i])
    elif i % 7 == 3:
        vagas.append(lista_elem[i][0])
    elif i % 7 == 4:
        banheiros.append(lista_elem[i][0])
    elif i % 7 == 5:
        andar.append(lista_elem[i])
    elif i % 7 == 6:
        mobilia.append(lista_elem[i])

converte_dias(lista_tempos)


Primeiro dataset com informações GERAIS.

In [34]:
descrição_imovel = {
    'ID': lista_id,
    'Tipo' : lista_tipos,
    'Descrição': lista_item,
    'Bairro' : lista_endereço1,
    'Logradouro' : lista_endereço2,
    'Publicação(Dias)' : lista_tempos_dias
}

df = pd.DataFrame(descrição_imovel)
df

,ID,Tipo,Descrição,Bairro,Logradouro,Publicação(Dias)
0,1781185,Apartamento,"Box, Varanda, Armários embutidos no quarto, Ar...","Méier, Rio de Janeiro",Rua Hermengarda,0.25
1,453438,Apartamento,"Box, Armários embutidos no quarto, Armários na...","Rio Comprido, Rio de Janeiro",Rua do Bispo,0.04
2,1762586,Casa,"Box, Varanda, Armários nos banheiros, Armários...","Tanque, Rio de Janeiro",Estrada da Covanca,1
3,1027119,Apartamento,"Piscina privativa, Quartos e corredores com po...","Tanque, Rio de Janeiro",Rua Manuel Vieira,4
4,939865,Apartamento,"Piscina privativa, Chuveiro a gás, Quartos e c...","Vila Isabel, Rio de Janeiro",Rua Luís Barbosa,5
...,...,...,...,...,...,...
67,1709213,Apartamento,"Box, Varanda, Armários embutidos no quarto, Ar...","Itanhangá, Rio de Janeiro",Avenida São Josemaria Escrivá,60
68,1690741,Apartamento,"Box, Varanda, Armários embutidos no quarto, Ar...","Pechincha, Rio de Janeiro",Rua Retiro dos Artistas,60
69,563680,Apartamento,"Box, Quintal, Área de serviço","Jardim Guanabara, Rio de Janeiro",Rua Marapeta,60
70,1781254,Casa,"Varanda, Armários embutidos no quarto, Armário...","Taquara, Rio de Janeiro",Rua André Rocha,2


 Segundo dataset com informações de VALORES

In [35]:
descrição_valores = {
    'ID': lista_id,
    'Preço' : lista_preços,
    'Condomínio' : lista_condominio,
    'IPTU' : lista_iptus,
}
df1 = pd.DataFrame(descrição_valores)
df1

,ID,Preço,Condomínio,IPTU
0,1781185,215.000,724,57
1,453438,360.000,750,0
2,1762586,450.000,70,0
3,1027119,150.000,450,0
4,939865,190.000,275,0
...,...,...,...,...
67,1709213,160.000,900,27
68,1690741,170.000,795,6
69,563680,410.000,600,241
70,1781254,250.000,0,0


Terceiro dataset com informações INTERNAS

In [36]:
descrição_especificas = {
    'ID': lista_id,
    'Quartos': quartos,
    'Área': tamanho,
    'Proximidade com Mêtro': metro,
    'Vagas': vagas,
    'Banheiros': banheiros,
    'Andares': andar,
    'Mobiliagem': mobilia
}

df2 = pd.DataFrame(descrição_especificas)
df2

,ID,Quartos,Área,Proximidade com Mêtro,Vagas,Banheiros,Andares,Mobiliagem
0,1781185,2,53,Próx. ao metrô,1,1,Até 3° andar,Sem mobília
1,453438,1,51,Próx. ao metrô,1,1,8° a 11° andar,Mobiliado
2,1762586,4,450,Sem metrô próx.,2,4,-,Sem mobília
3,1027119,2,55,Sem metrô próx.,1,1,Até 3° andar,Sem mobília
4,939865,2,78,Sem metrô próx.,-,1,Até 3° andar,Sem mobília
...,...,...,...,...,...,...,...,...
67,1709213,2,53,Sem metrô próx.,1,1,12° a 15° andar,Sem mobília
68,1690741,3,70,Sem metrô próx.,1,2,4° a 7° andar,Sem mobília
69,563680,3,140,Sem metrô próx.,2,3,Até 3° andar,Sem mobília
70,1781254,2,60,Sem metrô próx.,1,1,-,Mobiliado
